In [ ]:
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
from datetime import datetime, timedelta
import sys
sys.path.append('/Users/tarabaris/GitHub/odp-sdk-python/Examples')

## For SDK
from getpass import getpass
from odp_sdk import ODPClient
from getpass import getpass

## For plotting
from odp_sdk.utils.numeric import *
from odp_sdk.utils.visual import *

## Extra functions
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
sns.set_palette(sns.color_palette("bright", 47))
import warnings
warnings.filterwarnings("ignore")
import calendar

# Connect to API

In [ ]:
client = ODPClient(api_key=getpass(prompt='Insert your personal ODP API key:'), 
                       project="odp", client_name="odp")

# Pull data for desired lat/long and timespan
Don't forget to specify if you want flagged data included or excluded

In [ ]:
df=client.casts(longitude=[-10,35],
                latitude=[50,80],
                timespan=['2015-01-01','2018-12-31'],
                n_threads=35) 

In [ ]:
df.to_pickle('df2015_18_norway.pkl')

# Let's explore the data

In [ ]:
df2=df[(df['Temperature_WODflag']==0) & (df.z<50)][['externalId','lon','lat','z','datetime','Temperature']].dropna()

In [ ]:
df2.head()

In [ ]:
df_surf=interpolate_casts_to_z(df2,'Temperature',[0,10,20]) ##This can take a while

In [ ]:
df_surf.to_pickle('df_surf2015_18_norway.pkl')

In [ ]:
df_surf.head()

In [ ]:
df_surf.dropna(inplace=True)
df_surf['unixtime']=df_surf['datetime'].apply(lambda x : x.value) 
df_surf=df_surf[df_surf.Temperature>-1]
df_surf.describe()

# Plot temperature over months

In [ ]:
## Pick a year
df_2018 = df_surf[df_surf.datetime.dt.year == 2018]
coord = {
'month':[1,2,3,4,5,6,7,8,9,10,11,12],
'lat':[56,73,56],
'lon':[3, 20, 18],
'name':['North Sea','Barents Sea','Baltic Sea']}
kind = 'rbf'


temp = []
for mnth in coord['month']:
    try:
        points=df_2018[df_2018.datetime.dt.month == mnth][['lon','lat']].values.astype('float')
        values=df_2018[df_2018.datetime.dt.month == mnth]['Temperature'].values.astype('float')
        int_points=[coord['lon'], coord['lat']]
        grid,g=interpolate_to_grid(points.copy(),values.copy(),int_points.copy(), interp_type=kind,
                               rbf_func='linear',rbf_smooth=0.1,rescale=True)
        
    except:
        g=None
    temp.append(g)
    
df_temp = pd.DataFrame({
                    'month': [calendar.month_name[mnth] for mnth in coord['month']]})

temp_values = []
for i in range(len(coord['month'])):
    temp_values.append(temp[i].diagonal())

for name in coord['name']:
    temp_values = np.vstack(temp_values)
    df_temp[name] = temp_values[:,coord['name'].index(name)]
    
colors = sns.color_palette('bright')
plt.figure(figsize=(14,8))

for i, name in enumerate(coord['name']):
    plt.plot( 'month', name, data=df_temp, marker='o',color=colors[i], linewidth=2)
    plt.xlabel('Month')
    plt.ylabel('Temperature {}'.format(get_units()['Temperature']))
    plt.legend()
    
df_temp


# Plot seasonal temperature over years

In [ ]:
seasons = ['Winter', 'Winter', 'Winter', 'Spring', 'Spring', 'Spring','Summer', 'Summer','Summer','Fall', 'Fall', 'Fall', ]
month_to_season = dict(zip(range(1,13), seasons))

df_surf['season'] = df_surf.datetime.dt.month.map(month_to_season)

df_surf.head()

In [ ]:
season = 'Summer'
df_season = df_surf[df_surf.season == season]

coord = {
'year':[2015, 2016, 2017, 2018],
'lat':[56,73,56],
'lon':[3, 20, 18],
'name':['North Sea','Barents Sea','Baltic Sea']}
kind = 'rbf'


temp = []
for yr in coord['year']:
    try:
        points=df_season[df_season.datetime.dt.year == yr][['lon','lat']].values.astype('float')
        values=df_season[df_season.datetime.dt.year == yr]['Temperature'].values.astype('float')
        int_points=[coord['lon'], coord['lat']]
        grid,g=interpolate_to_grid(points.copy(),values.copy(),int_points.copy(), interp_type=kind,
                               rbf_func='linear',rbf_smooth=0.1,rescale=True)
        
    except:
#         print(mnth)
        g=None
    temp.append(g)
    
df_temp = pd.DataFrame({
                    'year': coord['year']})

temp_values = []
for i in range(len(coord['year'])):
    temp_values.append(temp[i].diagonal())

for name in coord['name']:
    temp_values = np.vstack(temp_values)
    df_temp[name] = temp_values[:,coord['name'].index(name)]
    
colors = sns.color_palette('bright')
plt.figure(figsize=(14,8))

for i, name in enumerate(coord['name']):
    plt.plot( 'year', name, data=df_temp, marker='o',color=colors[i], linewidth=2)
    plt.xlabel('Year')
    plt.xticks(coord['year'])
    plt.ylabel('Temperature {}'.format(get_units()['Temperature']))
    plt.legend()
    
df_temp


# Plot individual casts with depth

In [ ]:
cast_names=['cast_wod_3_2018_34350_19271220','cast_wod_3_2018_34350_19272097','cast_wod_3_2018_34350_19271373']
variable='Temperature'

plt.figure(figsize=(10,10))
legend=[]
for cast_name in cast_names:
    data=client.download_data_from_casts([cast_name])
    plt.plot(data[variable],data.z)
    plt.scatter(data[variable],data.z)
    legend.append('date: {}, {}'.format(data.date.iloc[0],cast_name))

plt.legend(legend,loc=0)
plt.gca().invert_yaxis()   
plt.ylabel('Depth (m)')
plt.xlabel(variable + ' ' + get_units()[variable])
